## Week 4, Lab 2: Predicting Chronic Kidney Disease in Patients
> Author: Matt Brems

We can sketch out the data science process as follows:
1. Define the problem.
2. Obtain the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

In this lab, we're going to focus on steps exploring data, building models and evaluating the models we build.

There are three links you may find important:
- [A set of chronic kidney disease (CKD) data and other biological factors](./chronic_kidney_disease_full.csv).
- [The CKD data dictionary](./chronic_kidney_disease_header.txt).
- [An article comparing the use of k-nearest neighbors and support vector machines on predicting CKD](./chronic_kidney_disease.pdf).

## Step 1: Define the problem.

Suppose you're working for Mayo Clinic, widely recognized to be the top hospital in the United States. In your work, you've overheard nurses and doctors discuss test results, then arrive at a conclusion as to whether or not someone has developed a particular disease or condition. For example, you might overhear something like:

> **Nurse**: Male 57 year-old patient presents with severe chest pain. FDP _(short for fibrin degradation product)_ was elevated at 13. We did an echo _(echocardiogram)_ and it was inconclusive.

> **Doctor**: What was his interarm BP? _(blood pressure)_

> **Nurse**: Systolic was 140 on the right; 110 on the left.

> **Doctor**: Dammit, it's an aortic dissection! Get to the OR _(operating room)_ now!

> _(intense music playing)_

In this fictitious but [Shonda Rhimes-esque](https://en.wikipedia.org/wiki/Shonda_Rhimes#Grey's_Anatomy,_Private_Practice,_Scandal_and_other_projects_with_ABC) scenario, you might imagine the doctor going through a series of steps like a [flowchart](https://en.wikipedia.org/wiki/Flowchart), or a series of if-this-then-that steps to diagnose a patient. The first steps made the doctor ask what the interarm blood pressure was. Because interarm blood pressure took on the values it took on, the doctor diagnosed the patient with an aortic dissection.

Your goal, as a research biostatistical data scientist at the nation's top hospital, is to develop a medical test that can improve upon our current diagnosis system for [chronic kidney disease (CKD)](https://www.mayoclinic.org/diseases-conditions/chronic-kidney-disease/symptoms-causes/syc-20354521).

**Real-world problem**: Develop a medical diagnosis test that is better than our current diagnosis system for CKD.

**Data science problem**: Develop a medical diagnosis test that reduces both the number of false positives and the number of false negatives.

---

## Step 2: Obtain the data.

### 1. Read in the data.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import patsy

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LogisticRegression

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
df = pd.read_csv('../4_02-lab-classification_model_evaluation/chronic_kidney_disease_full.csv')

### 2. Check out the data dictionary. What are a few features or relationships you might be interested in checking out?

In [4]:
df.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38.0,6000.0,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35.0,7300.0,4.6,no,no,no,good,no,no,ckd


In [5]:
df.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu',
       'sc', 'sod', 'pot', 'hemo', 'pcv', 'wbcc', 'rbcc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'class'],
      dtype='object')

---

## Step 3: Explore the data.

### 3. How much of the data is missing from each column?

In [6]:
df.isnull().sum()

age        9
bp        12
sg        47
al        46
su        49
rbc      152
pc        65
pcc        4
ba         4
bgr       44
bu        19
sc        17
sod       87
pot       88
hemo      52
pcv       71
wbcc     106
rbcc     131
htn        2
dm         2
cad        2
appet      1
pe         1
ane        1
class      0
dtype: int64

In [7]:
df_dropna = df.dropna()

In [8]:
df.shape

(400, 25)

In [9]:
df_dropna.shape

(158, 25)

### 4. Suppose that I dropped every row that contained at least one missing value. (In the context of analysis with missing data, we call this a "complete case analysis," because we keep only the complete cases!) How many rows would remain in our dataframe? What are at least two downsides to doing this?

> There's a good visual on slide 15 of [this deck](https://liberalarts.utexas.edu/prc/_files/cs/Missing-Data.pdf) that shows what a complete case analysis looks like if you're interested.

We have dropped more than 60% of our data neglecting other valid valuable values that might be crucial in examing the data for shaping into a 'complete case analysis'.

### 5. Thinking critically about how our data were gathered, it's likely that these records were gathered by doctors and nurses. Brainstorm three potential areas (in addition to the missing data we've already discussed) where this data might be inaccurate or imprecise.

1. Data values are subjected to humar error i.e inconsistent method of measuring across, inaccurate input to the systems
2. Data values are subjected to patients' discrepancy and accuracy
3. Data values might have been adjusted and rounded off to estimation for the ease of recording

---

## Step 4: Model the data.

### 6. Suppose that I want to construct a model where no person who has CKD will ever be told that they do not have CKD. What (very simple, no machine learning needed) model can I create that will never tell a person with CKD that they do not have CKD?

> Hint: Don't think about `statsmodels` or `scikit-learn` here.

Build a model that states that everyone they have CKD. This way, we can only commit False Positive error.

### 7. In problem 6, what common classification metric did we optimize for? Did we minimize false positives or negatives?

We optimized Sensitivity and minimized False Negatives.


### 8. Thinking ethically, what is at least one disadvantage to the model you described in problem 6?

We will be placing everyone who don't have CKD under unnecessary distress 

### 9. Suppose that I want to construct a model where a person who does not have CKD will ever be told that they do have CKD. What (very simple, no machine learning needed) model can I create that will accomplish this?

Build a model that states that everyone don't have CKD. This way, we can only commit False Negative error.

### 10. In problem 9, what common classification metric did we optimize for? Did we minimize false positives or negatives?

We optimized Specificity and minimized False Positives.

### 11. Thinking ethically, what is at least one disadvantage to the model you described in problem 9?

We will be placing everyone who actually do have CKD under in danger as they will go on with life untreated for their condition negatively affecting the chances of recovering and worsening their condition further.

### 12. Construct a logistic regression model in `sklearn` predicting class from the other variables. You may scale, select/drop, and engineer features as you wish - build a good model! Make sure, however, that you include at least one categorical/dummy feature and at least one quantitative feature.

> Hint: Remember to do a train/test split!

In [10]:
df.isnull().sum().sort_values(ascending = False)

rbc      152
rbcc     131
wbcc     106
pot       88
sod       87
pcv       71
pc        65
hemo      52
su        49
sg        47
al        46
bgr       44
bu        19
sc        17
bp        12
age        9
ba         4
pcc        4
htn        2
dm         2
cad        2
ane        1
appet      1
pe         1
class      0
dtype: int64

In [11]:
df.dtypes

age      float64
bp       float64
sg       float64
al       float64
su       float64
rbc       object
pc        object
pcc       object
ba        object
bgr      float64
bu       float64
sc       float64
sod      float64
pot      float64
hemo     float64
pcv      float64
wbcc     float64
rbcc     float64
htn       object
dm        object
cad       object
appet     object
pe        object
ane       object
class     object
dtype: object

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 25 columns):
age      391 non-null float64
bp       388 non-null float64
sg       353 non-null float64
al       354 non-null float64
su       351 non-null float64
rbc      248 non-null object
pc       335 non-null object
pcc      396 non-null object
ba       396 non-null object
bgr      356 non-null float64
bu       381 non-null float64
sc       383 non-null float64
sod      313 non-null float64
pot      312 non-null float64
hemo     348 non-null float64
pcv      329 non-null float64
wbcc     294 non-null float64
rbcc     269 non-null float64
htn      398 non-null object
dm       398 non-null object
cad      398 non-null object
appet    399 non-null object
pe       399 non-null object
ane      399 non-null object
class    400 non-null object
dtypes: float64(14), object(11)
memory usage: 78.2+ KB


In [13]:
df_copy = df

In [14]:
df_copy.replace(np.nan, 0, inplace=True)

In [15]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 25 columns):
age      400 non-null float64
bp       400 non-null float64
sg       400 non-null float64
al       400 non-null float64
su       400 non-null float64
rbc      400 non-null object
pc       400 non-null object
pcc      400 non-null object
ba       400 non-null object
bgr      400 non-null float64
bu       400 non-null float64
sc       400 non-null float64
sod      400 non-null float64
pot      400 non-null float64
hemo     400 non-null float64
pcv      400 non-null float64
wbcc     400 non-null float64
rbcc     400 non-null float64
htn      400 non-null object
dm       400 non-null object
cad      400 non-null object
appet    400 non-null object
pe       400 non-null object
ane      400 non-null object
class    400 non-null object
dtypes: float64(14), object(11)
memory usage: 78.2+ KB


In [16]:
cols = df_copy.columns

In [17]:
cols

Index(['age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu',
       'sc', 'sod', 'pot', 'hemo', 'pcv', 'wbcc', 'rbcc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'class'],
      dtype='object')

In [18]:
nonobject_cols = [col for col in df_copy.columns if df_copy[col].dtype != "object"]
print(nonobject_cols)

['age', 'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 'wbcc', 'rbcc']


In [19]:
nonobject_nominal_cols = ['al', 'su']

In [20]:
numerical_cols = list(set(nonobject_cols) - set(nonobject_nominal_cols))

In [21]:
numerical_cols

['sod',
 'pcv',
 'hemo',
 'bp',
 'sc',
 'pot',
 'wbcc',
 'rbcc',
 'age',
 'sg',
 'bu',
 'bgr']

In [22]:
nominal_cols = list(set(cols) - set(numerical_cols))

In [23]:
nominal_cols

['al',
 'dm',
 'pe',
 'cad',
 'pcc',
 'pc',
 'ba',
 'ane',
 'htn',
 'su',
 'appet',
 'rbc',
 'class']

In [24]:
#I will replace all the zero values to the alternative negative choice so as to increase the sensitivity.

In [25]:
df_copy['dm'].value_counts()

no     261
yes    137
0        2
Name: dm, dtype: int64

In [26]:
df_copy['dm'].replace(0, 'yes', inplace=True)

In [27]:
df_copy['su'].value_counts()

0.0    339
2.0     18
3.0     14
1.0     13
4.0     13
5.0      3
Name: su, dtype: int64

In [28]:
df_copy['class'].value_counts()

ckd       250
notckd    150
Name: class, dtype: int64

In [29]:
df_copy['al'].value_counts()

0.0    245
1.0     44
3.0     43
2.0     43
4.0     24
5.0      1
Name: al, dtype: int64

In [30]:
df_copy['ane'].value_counts()

no     339
yes     60
0        1
Name: ane, dtype: int64

In [31]:
df_copy['ane'].replace(0, 'yes', inplace=True)

In [32]:
df_copy['ba'].value_counts()

notpresent    374
present        22
0               4
Name: ba, dtype: int64

In [33]:
df_copy['ba'].replace(0, 'present', inplace=True)

In [34]:
df_copy['htn'].value_counts()

no     251
yes    147
0        2
Name: htn, dtype: int64

In [35]:
df_copy['htn'].replace(0, 'yes', inplace=True)

In [36]:
df_copy['rbc'].value_counts()

normal      201
0           152
abnormal     47
Name: rbc, dtype: int64

In [37]:
df_copy['pc'].value_counts()

normal      259
abnormal     76
0            65
Name: pc, dtype: int64

In [38]:
df_copy['pcc'].value_counts()

notpresent    354
present        42
0               4
Name: pcc, dtype: int64

In [39]:
df_copy['pcc'].replace(0, 'present', inplace=True)

In [40]:
df_copy['appet'].value_counts()

good    317
poor     82
0         1
Name: appet, dtype: int64

In [41]:
df_copy['appet'].replace(0, 'poor', inplace=True)

In [42]:
df_copy['cad'].value_counts()

no     364
yes     34
0        2
Name: cad, dtype: int64

In [43]:
df_copy['cad'].replace(0, 'yes', inplace=True)

In [44]:
df_copy['pe'].value_counts()

no     323
yes     76
0        1
Name: pe, dtype: int64

In [45]:
df_copy['pe'].replace(0, 'yes', inplace=True)

In [46]:
#Dropping 
df_copy.drop(['rbc','pc','sod','pot','pcv','wbcc','rbcc'], axis=1, inplace=True)

In [47]:
df_copy.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'pcc', 'ba', 'bgr', 'bu', 'sc', 'hemo',
       'htn', 'dm', 'cad', 'appet', 'pe', 'ane', 'class'],
      dtype='object')

In [48]:
df_copy.isnull().sum()

age      0
bp       0
sg       0
al       0
su       0
pcc      0
ba       0
bgr      0
bu       0
sc       0
hemo     0
htn      0
dm       0
cad      0
appet    0
pe       0
ane      0
class    0
dtype: int64

### Creating Dummies

In [49]:
df_copy_dummy = pd.get_dummies(data=df_copy, drop_first=True)

In [50]:
df_copy_dummy.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc', 'hemo', 'pcc_present',
       'ba_present', 'htn_yes', 'dm_yes', 'cad_yes', 'appet_poor', 'pe_yes',
       'ane_yes', 'class_notckd'],
      dtype='object')

In [51]:
df_copy_dummy.drop('class_notckd', axis=1, inplace=True)

In [52]:
df_copy_dummy ['class'] = df_copy['class']

In [53]:
df_copy_dummy.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc', 'hemo', 'pcc_present',
       'ba_present', 'htn_yes', 'dm_yes', 'cad_yes', 'appet_poor', 'pe_yes',
       'ane_yes', 'class'],
      dtype='object')

In [54]:
col_list = list(df_copy_dummy.columns)
features = []
[features.append(col) for col in col_list if col != 'class']

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [55]:
X = df_copy_dummy[features]
y = df_copy_dummy['class']

In [56]:
poly = PolynomialFeatures(include_bias=False, degree=2)

In [57]:
X_poly = poly.fit_transform(X)

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, random_state = 42)

In [59]:
ss = StandardScaler()

In [60]:
ss.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [61]:
X_train_ss = ss.transform(X_train)
X_test_ss = ss.transform(X_test)

In [62]:
logreg = LogisticRegression()

In [64]:
logreg.fit(X_train_ss, y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [65]:
logreg.score(X_train_ss, y_train)

0.9666666666666667

In [66]:
logreg.score(X_test_ss, y_test)

0.96

---

## Step 5: Evaluate the model.

### 13. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your quantitative features.

In [67]:
coefficients = pd.concat([pd.DataFrame(X.columns),pd.DataFrame(np.transpose(logreg.coef_))], axis = 1)
coefficients.sort_values

<bound method DataFrame.sort_values of        0         0
0    age  0.540851
1     bp  0.558110
2     sg  0.219941
3     al -0.612890
4     su -0.068841
..   ...       ...
165  NaN  0.204535
166  NaN  0.160135
167  NaN -0.257762
168  NaN  0.071004
169  NaN -0.164064

[170 rows x 2 columns]>

### 14. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your categorical/dummy features.

### 15. Despite being a relatively simple model, logistic regression is very widely used in the real world. Why do you think that's the case? Name at least two advantages to using logistic regression as a modeling technique.

1.  Logistic Regression will predict continuous values.
2.  Logistic Regression connects regression and classification.
3.  Logistic Regression can predict probabilities.

### 16. Does it make sense to generate a confusion matrix on our training data or our test data? Why? Generate it on the proper data.

> Hint: Once you've generated your predicted $y$ values and you have your observed $y$ values, then it will be easy to [generate a confusion matrix using sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).

In [68]:
predictions = logreg.predict(X_test_ss)

In [69]:
cm = confusion_matrix(y_test, predictions)

In [70]:
cm

array([[62,  3],
       [ 1, 34]])

In [71]:
cm = pd.DataFrame(cm, columns=['Predicted Negative','Predicted Positive'], index=['Actual Negative','Actual Positive'])

In [72]:
cm

,Predicted Negative,Predicted Positive
Actual Negative,62,3
Actual Positive,1,34


### 17. In this hospital case, we want to predict CKD. Do we want to optimize for sensitivity, specificity, or something else? Why? (If you don't think there's one clear answer, that's okay! There rarely is. Be sure to defend your conclusion!)

In medical context, we should optimize sensitivity as it is better to get more false positives than false negatives. If patient gets a false positive, a series of follow up tests and appointments will show otherwise. However if a patient gets false negative, he/she will move on from the hospital and neglect themselves from immediate care until it is too late.

### 18 (BONUS). Write a function that will create an ROC curve for you, then plot the ROC curve.

Here's a strategy you might consider:
1. In order to even begin, you'll need some fit model. Use your logistic regression model from problem 12.
2. We want to look at all values of your "threshold" - that is, anything where .predict() gives you above your threshold falls in the "positive class," and anything that is below your threshold falls in the "negative class." Start the threshold at 0.
3. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
4. Increment your threshold by some "step." Maybe set your step to be 0.01, or even smaller.
5. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
6. Repeat steps 3 and 4 until you get to the threshold of 1.
7. Plot the values of sensitivity and 1 - specificity.

### 19. Suppose you're speaking with the biostatistics lead at Mayo Clinic, who asks you "Why are unbalanced classes generally a problem? Are they a problem in this particular CKD analysis?" How would you respond?

### 20. Suppose you're speaking with a doctor at Mayo Clinic who, despite being very smart, doesn't know much about data science or statistics. How would you explain why unbalanced classes are generally a problem to this doctor?

### 21. Let's create very unbalanced classes just for the sake of this example! Generate very unbalanced classes by [bootstrapping](http://stattrek.com/statistics/dictionary.aspx?definition=sampling_with_replacement) (a.k.a. random sampling with replacement) the majority class.

1. The majority class are those individuals with CKD.
2. Generate a random sample of size 200,000 of individuals who have CKD **with replacement**. (Consider setting a random seed for this part!)
3. Create a new dataframe with the original data plus this random sample of data.
4. Now we should have a dataset with around 200,000 observations, of which only about 0.00075% are non-CKD individuals.

In [73]:
df_copy_dummy.shape

(400, 18)

In [74]:
df_copy_dummy['class'].value_counts()

ckd       250
notckd    150
Name: class, dtype: int64

In [75]:
df_copy_maj = df_copy_dummy[ df_copy_dummy['class'] == 'ckd' ]
df_copy_min = df_copy_dummy[ df_copy_dummy['class'] == 'notckd' ]

In [76]:
from sklearn.utils import resample

In [77]:
df_upsample = resample(df_copy_maj, replace = True, n_samples = 200000)

In [78]:
df_upsample = pd.concat([df_upsample, df_copy_min])

In [79]:
df_upsample['class'].value_counts()

ckd       200000
notckd       150
Name: class, dtype: int64

### 22. Build a logistic regression model on the unbalanced class data and evaluate its performance using whatever method(s) you see fit. How would you describe the impact of unbalanced classes on logistic regression as a classifier?
> Be sure to look at how well it performs on non-CKD data.

In [80]:
X = df_upsample[features]
y = df_upsample['class']

In [81]:
X_poly = poly.fit_transform(X)

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X_poly, y)

In [83]:
ss.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [84]:
X_train_ss = ss.transform(X_train)
X_test_ss = ss.transform(X_test)

In [85]:
logreg.fit(X_train_ss, y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [86]:
logreg.score(X_train_ss, y_train)

0.9997468556810915

In [87]:
predictions = logreg.predict(X_test_ss)

In [88]:
cm_boots = confusion_matrix(y_test, predictions)

In [89]:
cm_boots = pd.DataFrame(cm_boots, columns=['Predicted Negative','Predicted Positive'], 
                       index=['Actual Negative','Actual Positive'])

In [90]:
cm_boots

,Predicted Negative,Predicted Positive
Actual Negative,49993,0
Actual Positive,22,23


There are no False Positive now.

---

## Step 6: Answer the problem.

At this step, you would generally answer the problem! In this situation, you would likely present your model to doctors or administrators at the hospital and show how your model results in reduced false positives/false negatives. Next steps would be to find a way to roll this model and its conclusions out across the hospital so that the outcomes of patients with CKD (and without CKD!) can be improved!